In [33]:
# Unsloth 라이브러리에서 FastLanguageModel을 임포트
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # 최대 시퀀스 길이를 설정 ( 텍스트의 최대 길이를 지정)
dtype = None  # 자동 감지를 위해 None 설정. Tesla T4는 Float16, Ampere+는 Bfloat16 사용. 모델의 파라미터를 저장할 데이터 타입
load_in_4bit = True  # 메모리 사용량을 줄이기 위해 4비트 양자화 사용. 다만 양자화에 따른 손실이 있어서 필요에 따라 False로 설정 가능

# 사전 학습된 모델과 토크나이저 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",  # 사용할 모델 이름
    max_seq_length = max_seq_length,         # 설정한 최대 시퀀스 길이
    dtype = dtype,                           # 데이터 타입 설정
    load_in_4bit = load_in_4bit,             # 4비트 양자화 여부
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2+cu118. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [34]:
# PEFT(파라미터 효율적 파인튜닝) 모델 설정
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA 랭크 설정. 8, 16, 32, 64, 128 권장. r 값이 클수록 모델이 더 많은 정보를 학습할 수 있지만, 너무 크면 메모리를 많이 사용
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],  # PEFT 적용할 모듈 목록. 모델의 특정 부분(모듈)에만 학습
    lora_alpha = 16,        # LoRA 알파 설정 LoRA라는 기술이 얼마나 강하게 작용할지 조절
    lora_dropout = 0,       # LoRA 드롭아웃 설정. 0으로 최적화
    bias = "none",          # 바이어스 설정. "none"으로 최적화

    # "unsloth" 사용 시 VRAM 절약 및 배치 사이즈 2배 증가
    # 학습할 때 메모리를 절약하는 방법을 사용하는 설정
    use_gradient_checkpointing = "unsloth",  # 매우 긴 컨텍스트를 위해 "unsloth" 설정
    random_state = 3407,    # 랜덤 시드 설정
    use_rslora = False,     # 랭크 안정화 LoRA 사용 여부
    loftq_config = None,    # LoftQ 설정 (사용하지 않음)
)

In [35]:
# 모델에게 주어질 텍스트의 형식을 정의
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# 맨윗줄: 모델에게 앞으로 제공될 지시사항을 기반으로 적절한 응답을 작성하라고 말함.
# 모델이 어떤 작업을 수행해야 하는지 명확히 이해할 수 있도록 도와줌
# Instruction:은 지시사항이 제공될 부분
# Response:는 모델이 생성해야 할 응답이 제공될 부분

# EOS 토큰 가져오기 (생성 종료를 위해 필요)
EOS_TOKEN = tokenizer.eos_token  # 반드시 EOS_TOKEN을 추가해야 함
EOS_TOKEN

'<|end_of_text|>'

In [36]:
# 프롬프트 포맷팅 함수 정의
def formatting_prompts_func(examples):
    instructions = examples["question"]  # 데이터셋의 'instruction' 필드
    outputs      = examples["answer"]       # 데이터셋의 'output' 필드
    texts = []
    for instruction, output in zip(instructions, outputs):
                                                           # EOS_TOKEN을 추가하지 않으면 생성이 무한히 계속됨
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN  # 프롬프트 형식에 맞게 텍스트 생성
        texts.append(text)
    return { "text" : texts, }  # 'text' 필드로 반환

from datasets import load_dataset  # Hugging Face datasets 라이브러리 임포트

# 데이터셋 로드 (건설 안전 RAFT 데이터셋)
dataset = load_dataset("YoungjaeDev/construction-safety-raft-kiwi-bm25-faiss-37")

# 프롬프트 포맷팅 함수 적용하여 데이터셋 변환
dataset = dataset.map(formatting_prompts_func, batched = True,)

# from datasets import load_dataset
# dataset = load_dataset( "csv", data_files = "/home/alpaco/chat_bot/wellnis.csv", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched=True)

# 예제 엑셀 형태로 데이터셋을 만든다면?
# from datasets import load_dataset
# dataset = load_dataset( "csv", data_files = "data.csv", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched=True)
#
#

Repo card metadata block was not found. Setting CardData to empty.


In [37]:
dataset['train'][:5]

{'question': ["교량공사에서 버팀보 관련 '해체작업' 중에 끼임가 발생했습니다. 사고원인은 '해체 후 부재를 철거하여 작업구 외부로 인양하기 전 주변지반을 평탄하게 정리한 후, 본 해체 인양작업을 진행해야하나 주변지반을 정리하지 않고, 경사진 곳에 자재를 적치하다가 발생된 사고임.'입니다. 이러한 사고의 재발을 방지하기 위한 대책과 향후 조치 계획을 알려주세요.",
  "수장공사에서 기중기(이동식크레인 등) 관련 '운반작업' 중에 떨어짐(2미터 미만)가 발생했습니다. 사고원인은 '작업개시 전 신호수 및 작업관리자가 안전관리자에게 현장확인 요청 협의하는 과정에서 사고자가 운전석 아웃트리거가 펴지지 않은 것을 미인지한 상태로 작업 개시'입니다. 이러한 사고의 재발을 방지하기 위한 대책과 향후 조치 계획을 알려주세요.",
  "건축물 부대공사에서 시스템동바리 관련 '운반작업' 중에 깔림가 발생했습니다. 사고원인은 '적재한 비계를 꺼내면서'입니다. 이러한 사고의 재발을 방지하기 위한 대책과 향후 조치 계획을 알려주세요.",
  "철근콘크리트공사에서 자재 관련 '설치작업' 중에 안전사고가 발생했습니다. 사고원인은 '남웅건설 김00씨가 형틀작업으로 인해 허리에 부담되는 작업을 진행하였고 이에 요추부 추간판탈출증을 진단받았다고 요양급여 신청'입니다. 이러한 사고의 재발을 방지하기 위한 대책과 향후 조치 계획을 알려주세요.",
  "철근콘크리트공사에서 콘크리트살포기 관련 '타설작업' 중에 끼임가 발생했습니다. 사고원인은 '콘크리트 분배기 회전부위 레일을 손으로 짚어 사고 발생'입니다. 이러한 사고의 재발을 방지하기 위한 대책과 향후 조치 계획을 알려주세요."],
 'context': ['아래 참고 자료를 검토하고, 도움이 될 경우에만 활용하여 답변해주세요. 참고 자료가 질문과 관련이 없거나 부적절한 경우 자체 지식을 바탕으로 답변해주세요:\n\n문서 1: (2) 해체작업순서는 설치순서의 역순으로 실시하여야 한다.\n\n(3) 후방 가로보, 추진코, 가로보 등의 부

In [38]:
## 학습 설정
from trl import SFTTrainer  # TRL 라이브러리에서 SFTTrainer 임포트
from transformers import TrainingArguments  # 트랜스포머 라이브러리에서 TrainingArguments 임포트
from unsloth import is_bfloat16_supported  # BFloat16 지원 여부 확인 함수 임포트

# SFTTrainer 인스턴스 생성
trainer = SFTTrainer(
    model = model,                           # 학습할 모델
    tokenizer = tokenizer,                   # 사용할 토크나이저
    train_dataset = dataset['train'],                 # 학습할 데이터셋 ★★★★★★★★
    dataset_text_field = "text",             # 데이터셋의 텍스트 필드 이름 ★★★★★★★★
    max_seq_length = max_seq_length,         # 최대 시퀀스 길이
    dataset_num_proc = 2,                    # 데이터셋 전처리에 사용할 프로세스 수 cpu
    packing = False,                         # 짧은 시퀀스의 경우 packing을 비활성화 (학습 속도 5배 향상 가능)
    args = TrainingArguments(
        per_device_train_batch_size = 2,     # 디바이스 당 배치 사이즈
        gradient_accumulation_steps = 4,     # 그래디언트 누적 단계 수
        warmup_steps = 5,                     # 워밍업 스텝 수
        # num_train_epochs = 1,               # 전체 학습 에폭 수 설정 가능
        max_steps = 60,                       # 최대 학습 스텝 수
        learning_rate = 2e-4,                 # 학습률
        fp16 = not is_bfloat16_supported(),   # BFloat16 지원 여부에 따라 FP16 사용
        bf16 = is_bfloat16_supported(),       # BFloat16 사용 여부
        logging_steps = 1,                    # 로깅 빈도
        optim = "adamw_8bit",                  # 옵티마이저 설정 (8비트 AdamW)
        weight_decay = 0.01,                  # 가중치 감쇠
        lr_scheduler_type = "linear",         # 학습률 스케줄러 타입
        seed = 3407,                           # 랜덤 시드 설정
        output_dir = "outputs",                # 출력 디렉토리
    ),
)

## 학습 실행
trainer_stats = trainer.train()  # 모델 학습 시작 # 3d6e4b9a0ddcd9edfafad59dcde0fd8c666954fd

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 22,250 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,3.180000
2,3.042100
3,3.074400
4,3.052000
5,3.075800
6,2.705400
7,2.321400
8,2.167700
9,2.072400
10,2.063900


In [39]:
# 모델 저장 로컬폴더에다가 저장하는 방식
model.save_pretrained("construction")  # Local saving
tokenizer.save_pretrained("construction")

# 이 이외에 허깅페이스나 다른 hub에 push해서 저장하는 방법이 있음
# 다만, 업로드 속도와 다운로드 속도를 고려해야함.

('construction/tokenizer_config.json',
 'construction/special_tokens_map.json',
 'construction/tokenizer.json')

In [40]:
from unsloth import FastLanguageModel
import torch

# 저장된 경로 지정
save_directory = "/home/alpaco/chat_store/construction"  # 모델이 저장된 디렉토리 경로

# 모델과 토크나이저 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_directory,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,  # 양자화 옵션을 동일하게 설정
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2+cu118. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [42]:
# 추론해보기
FastLanguageModel.for_inference(model)  # 네이티브 2배 빠른 추론 활성화

# 추론을 위한 입력 준비
inputs = tokenizer(
[
    alpaca_prompt.format(
        "교량공사에서 버팀보 관련 '해체작업' 중에 끼임가 발생했습니다. 사고원인은 '해체 후 부재를 철거하여 작업구 외부로 인양하기 전 주변지반을 평탄하게 정리한 후, 본 해체 인양작업을 진행해야하나 주변지반을 정리하지 않고, 경사진 곳에 자재를 적치하다가 발생된 사고임.'입니다. 이러한 사고의 재발을 방지하기 위한 대책과 향후 조치 계획을 알려주세요.", # 인스트럭션 (명령어)
        "", # 출력 - 생성할 답변을 비워둠
    )
], return_tensors = "pt").to("cuda")  # 텐서를 PyTorch 형식으로 변환하고 GPU로 이동

from transformers import TextStreamer  # 텍스트 스트리밍을 위한 TextStreamer 임포트

text_streamer = TextStreamer(tokenizer)  # 토크나이저를 사용하여 스트리머 초기화

# 모델을 사용하여 텍스트 생성 및 스트리밍 출력
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)  # 최대 128개의 새로운 토큰 생성

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
교량공사에서 버팀보 관련 '해체작업' 중에 끼임가 발생했습니다. 사고원인은 '해체 후 부재를 철거하여 작업구 외부로 인양하기 전 주변지반을 평탄하게 정리한 후, 본 해체 인양작업을 진행해야하나 주변지반을 정리하지 않고, 경사진 곳에 자재를 적치하다가 발생된 사고임.'입니다. 이러한 사고의 재발을 방지하기 위한 대책과 향후 조치 계획을 알려주세요.

### Response:
해체작업 전 주변지반 정리와 안전교육 철저를 통한 재발 방지 대책.<|end_of_text|>


In [44]:
# 답변만 추출
generated_text = tokenizer.decode(_[0], skip_special_tokens=True)
generated_text.split("### Response:")[1].strip()

'해체작업 전 주변지반 정리와 안전교육 철저를 통한 재발 방지 대책.'